In [159]:
import os
import csv
import time
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import nltk

## Deal with ocr data

In [160]:
df_ocr=pd.read_csv("../dataset/rawdata/ocr.csv")
df_ocr=df_ocr.drop(columns=['humor','com_score'])
df_ocr.rename(columns={'U':'User','Humorous_reply':'Humorous reply'},inplace = True)
df_ocr

,User,Humorous reply
0,WHY ARE YOU TALKING IN THE MIDDLE OF MY CLASS?,WHY ARE YOU TEACHING IN THE MIDDLE OF MY CONVE...
1,Hey long time no talk,let's keep it that way
2,what happens to the car if you press the brake...,it takes a screenshot
3,WHAT EVENT IS MADONNA ATTENDING HERE?,PARENT TEACHER NIGHT
4,"My friend : Bro, ""your mom"" jokes are SO overused","Me saying ""just like your mom"""
...,...,...
311,I’m in a lot of pain and could use some light ...,Do you know the one about the blind hooker?
312,I’m going out tomorrow night with some friends...,new ice age?
313,Why do you pick someone up off the ground when...,i guess i get a little carried away.
314,What's your unique talent?,"I've had people start looking around, trying t..."


## Deal with train345k

In [161]:
df_160=pd.read_csv("../dataset/rawdata/train160k.csv")
df_160=df_160[df_160['humor']==True]
df_160=df_160[df_160['text'].apply(lambda x: len(nltk.sent_tokenize(x))==2)]
df_160

,text,humor
2,What do you call a turtle without its shell? d...,True
6,What is a pokemon master's favorite kind of pa...,True
7,Why do native americans hate it when it rains ...,True
13,How are music and candy similar? we throw away...,True
21,Dating tip: surprise your date! show up a day ...,True
...,...,...
159985,Why did the chicken cross the road? because pa...,True
159986,"If a hobbit were to bake a hairstyle, what wou...",True
159988,How do you know that there's a monster in your...,True
159991,Why are chefs the meanest? because they beat t...,True


In [162]:
df_345=pd.read_csv("../dataset/rawdata/train345k.tsv",header=None,usecols=[0,4], names=['text', 'humor'])
df_345['text']=df_345['text'].apply(lambda x:x.split('\t')[1])
df_345=df_345[df_345['text'].apply(lambda x: len(nltk.sent_tokenize(x))==2)]
df_345

,text,humor
0,How do you end two deaf persons' arguing? Swit...,NaN
1,What does a blonde have in common with a hurri...,NaN
2,I went to a white pride parade the other day. ...,NaN
6,I'd make a joke about my neighbor.... But it w...,NaN
10,What’s your favourite insult joke? I’ll get it...,NaN
...,...,...
342431,What do you call a low-quality golf course? Su...,NaN
342432,Potassium texted Sodium asking to go for coffe...,NaN
342433,Where does the General keep his armies? In his...,NaN
342434,"I like to do the ""drink till she's cute thing""...",NaN


In [163]:
df_all=df_160.append(df_345)
df_all=df_all.drop_duplicates(subset=['text'])
df_all

,text,humor
2,What do you call a turtle without its shell? d...,True
6,What is a pokemon master's favorite kind of pa...,True
7,Why do native americans hate it when it rains ...,True
13,How are music and candy similar? we throw away...,True
21,Dating tip: surprise your date! show up a day ...,True
...,...,...
342426,Why did Kawhi choose the Clippers instead of t...,NaN
342431,What do you call a low-quality golf course? Su...,NaN
342432,Potassium texted Sodium asking to go for coffe...,NaN
342434,"I like to do the ""drink till she's cute thing""...",NaN


In [164]:
df_all['User']=" "
df_all['Humorous reply']=" "
df_all['User']=df_all['text'].apply(lambda x:nltk.sent_tokenize(x)[0])
df_all['Humorous reply']=df_all['text'].apply(lambda x:nltk.sent_tokenize(x)[1])

In [165]:
df_all=df_all.drop(columns=['text','humor'])

In [166]:
df_all=df_all.append(df_ocr)
df_all

,User,Humorous reply
2,What do you call a turtle without its shell?,dead.
6,What is a pokemon master's favorite kind of pa...,wartortellini!
7,Why do native americans hate it when it rains ...,because it brings mayflowers.
13,How are music and candy similar?,we throw away the rappers.
21,Dating tip: surprise your date!,show up a day early.
...,...,...
311,I’m in a lot of pain and could use some light ...,Do you know the one about the blind hooker?
312,I’m going out tomorrow night with some friends...,new ice age?
313,Why do you pick someone up off the ground when...,i guess i get a little carried away.
314,What's your unique talent?,"I've had people start looking around, trying t..."


In [167]:
df_all=df_all.drop_duplicates(subset=['User'])
df_all['User']=df_all['User'].apply(lambda x:"User: "+x)
df_all['Humorous reply']=df_all['Humorous reply'].apply(lambda x:"Humorous reply: "+x)

/Users/johnchen/opt/anaconda3/envs/SP/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/johnchen/opt/anaconda3/envs/SP/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [168]:
df_all = df_all.sample(frac=1).reset_index(drop=True)
df_all['combined']=df_all['User']+df_all['Humorous reply']
df_all['length']=df_all['combined'].apply(lambda x:len(x))
df_all=df_all[df_all['length']<=430]
df_all

,User,Humorous reply,combined,length
0,User: Do you wanna know what my epileptic cous...,Humorous reply: The Worm,User: Do you wanna know what my epileptic cous...,94
1,User: A farmer has 96 cows.,"Humorous reply: when he rounds them up, he has...",User: A farmer has 96 cows.Humorous reply: whe...,84
2,User: How do you keep a running back from drop...,Humorous reply: Paint it like a watermelon.,User: How do you keep a running back from drop...,107
3,User: What do you call an angry cashew?,Humorous reply: a pissed-achio.,User: What do you call an angry cashew?Humorou...,70
4,User: Why didn't the mummy go on vacation?,Humorous reply: He was afraid he'd relax and u...,User: Why didn't the mummy go on vacation?Humo...,94
...,...,...,...,...
198371,User: What did the Eggo waffle say while being...,Humorous reply: Leggo’a me!,User: What did the Eggo waffle say while being...,98
198372,User: How often does an oriental farmer milk h...,Humorous reply: dairy,User: How often does an oriental farmer milk h...,75
198373,"User: Vladimir Putin: ""You say Ukraine.""","Humorous reply: ""I say MYkraine!""","User: Vladimir Putin: ""You say Ukraine.""Humoro...",73
198374,User: Three friends on a bike There's three fr...,Humorous reply: Let's call them Michael,User: Three friends on a bike There's three fr...,168


In [169]:
my_data=df_all
train_size = 0.7
validate_size = 0.2
train, validate, test = np.split(my_data.sample(frac=1), [int(train_size * len(my_data)), int((validate_size + train_size) * len(my_data))])

In [170]:
train

,User,Humorous reply,combined,length
155851,User: The Aggie comes home from work...,Humorous reply: He hangs up his coat,User: The Aggie comes home from work...Humorou...,75
113474,User: My family is furious at me for implying ...,Humorous reply: I suppose the funeral wasn’t t...,User: My family is furious at me for implying ...,171
148004,User: A joke died on stage.,Humorous reply: It leaves behind an orphaned p...,User: A joke died on stage.Humorous reply: It ...,82
44217,User: You know dislexic kids must gave terribl...,Humorous reply: Also satan must get a lot of m...,User: You know dislexic kids must gave terribl...,110
14442,User: Why did the piglets do badly in school?,Humorous reply: they were all slow loiners.,User: Why did the piglets do badly in school?H...,88
...,...,...,...,...
54323,User: What's the difference between a weed and...,Humorous reply: One is a dandelion and one is ...,User: What's the difference between a weed and...,121
50678,User: Which honorary degree is to be awarded t...,Humorous reply: Ph.P.,User: Which honorary degree is to be awarded t...,112
126516,User: A priest and a permanently drunken bus d...,"Humorous reply: St. Peter says to the priest ""...",User: A priest and a permanently drunken bus d...,192
162272,User: My grandmom said i had a man's nose when...,Humorous reply: Considering they never stop gr...,User: My grandmom said i had a man's nose when...,110


In [171]:
validate

,User,Humorous reply,combined,length
15872,User: What did the Middle East say when they s...,Humorous reply: Oman,User: What did the Middle East say when they s...,100
106356,User: What did the Nazi say to the pregnant Jew?,Humorous reply: I see you have another Jew in ...,User: What did the Nazi say to the pregnant Je...,103
78410,User: What did the hobo get for christmas?,Humorous reply: nothing.,User: What did the hobo get for christmas?Humo...,66
82624,User: What do you get when you cross an elepha...,Humorous reply: a pachydermatologist.,User: What do you get when you cross an elepha...,104
153113,User: What did the dyslexic German event plann...,Humorous reply: The hall of cost will be 1945,User: What did the dyslexic German event plann...,98
...,...,...,...,...
190541,User: My sister asked for me to bring her some...,Humorous reply: It's pretty fucking hard to wr...,User: My sister asked for me to bring her some...,162
39915,User: What's the first thing that goes through...,Humorous reply: Usually a bullet,User: What's the first thing that goes through...,125
74561,User: The Boston symphony orchestra was playin...,Humorous reply: So,User: The Boston symphony orchestra was playin...,207
85225,User: Why can't the incredible hulk find a gir...,Humorous reply: because all the girls know he ...,User: Why can't the incredible hulk find a gir...,119


In [172]:
test

,User,Humorous reply,combined,length
171463,User: My friend told me that he could create a...,Humorous reply: I said,User: My friend told me that he could create a...,124
44270,User: An electron is driving down the highway....,Humorous reply: The policeman says,User: An electron is driving down the highway....,117
66344,User: What do you call a group of Turkeys?,Humorous reply: A Turkey Club,User: What do you call a group of Turkeys?Humo...,71
112212,User: Why are graveyards so noisy?,Humorous reply: because of all of the coffin.,User: Why are graveyards so noisy?Humorous rep...,79
182342,User: Barack Obama's vice president wants to m...,Humorous reply: Obama: Joe,User: Barack Obama's vice president wants to m...,89
...,...,...,...,...
101601,User: Did you know that Helen Keller lived in ...,Humorous reply: Neither did she,User: Did you know that Helen Keller lived in ...,92
26493,User: What do you call an asian pothead campin...,Humorous reply: a yellowstoner.,User: What do you call an asian pothead campin...,94
149821,User: What are unicorns who want better workin...,Humorous reply: unioncorns.,User: What are unicorns who want better workin...,114
8780,User: I went to the zoo today but there was on...,Humorous reply: It was a Shitzu,User: I went to the zoo today but there was on...,91


In [173]:
file1 = open("../dataset/train/train.txt","w")
for index, row in train.iterrows():
    str1=row['User']
    str2=row['Humorous reply']
    file1.write(str1)
    file1.write('\n')
    file1.write(str2)
    file1.write('<|endoftext|>')
file1.close

<function TextIOWrapper.close()>

In [174]:
file2 = open("../dataset/val/val.txt","w")
for index, row in validate.iterrows():
    str1=row['User']
    str2=row['Humorous reply']
    file2.write(str1)
    file2.write('\n')
    file2.write(str2)
    file2.write('<|endoftext|>')
file2.close

<function TextIOWrapper.close()>

In [179]:
# file3 = open("../dataset/test/test.txt","w")
file_count=1
count=0
for index, row in test.iterrows():
    with open(f'../dataset/test/test_{file_count}.txt',"a") as f:
        str1=row['User']
        str2=row['Humorous reply']
        f.write(str1)
        f.write('\n')
        f.write(str2)
        f.write('<|endoftext|>')
        count=count+1
        if count % 5000==0:
            file_count=file_count+1
            f.close

In [176]:
file4 = open("../dataset/test_user.txt","w")
for index, row in test.iterrows():
    str1=row['User']
#     str2=row['Humorous reply']
    file4.write(str1)
    file4.write('\n')
#     file4.write('Humorous reply: ')
file4.close






<function TextIOWrapper.close()>

In [158]:
from transformers import GPT2TokenizerFast


GPT2TokenizerFast.max_model_input_sizes['gpt2'] = 1e20  # disables a misleading warning
encoder = GPT2TokenizerFast.from_pretrained('gpt2')
test=encoder.encode(str_check)

len(test)

110